# OpenAI-SDK Demo

Using the responses API:
<p align="center">
    <img src="..\assets\responses_openai_api.png">
<p>

For more info the documentation is [here](https://platform.openai.com/docs/guides/migrate-to-responses).

## First just normal text generation

In [10]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI()

response = client.responses.create(
    model="gpt-3.5-turbo",
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)

Once upon a time, a kind-hearted unicorn spread glittering dreams through the night sky, gently lulling all to sleep with its magical presence.


### LLMs with Multimodal Input Demo

Multimodal input I'm going to be testing first is images + text.

Supported image formats: ['image/jpeg', 'image/png', 'image/gif', 'image/webp']

In [13]:
prompt ="What is happening in this photo?"
image_url =r"https://ca-times.brightspotcdn.com/dims4/default/2f806f3/2147483647/strip/true/crop/4148x2941+0+0/resize/1200x851!/quality/75/?url=https%3A%2F%2Fcalifornia-times-brightspot.s3.amazonaws.com%2F37%2F80%2Fe270fcc94a17b660d564ad2a8bd4%2Fap20168042659150.jpg"
# This photo depicts Obama and Biden

response = client.responses.create(
    model = "gpt-4o", #only certain models do multimodal inputs
    input = [
        {"role": "user",
         "content": [
             {"type": "input_text", "text": prompt},
             {"type": "input_image", "image_url": image_url}
            ]
        }
    ]
)

print(response.output_text)

# LIMITATIONS
# Asked who is in this photo and it didn't know

In the photo, one person is speaking at a podium with the presidential seal. Another person stands to the side, looking thoughtful with a hand on their chin. This appears to be a formal event or press conference.


In [17]:
prompt ="What company's logo is this?"
image_url =r"https://www.datocms-assets.com/104397/1714381237-datadog-logo-with-white-background.png"
# This photo is the datadog logo

response = client.responses.create(
    model = "gpt-4o", #only certain models do multimodal inputs
    input = [
        {"role": "user",
         "content": [
             {"type": "input_text", "text": prompt},
             {"type": "input_image", "image_url": image_url}
            ]
        }
    ]
)

print(response.output_text)

This is the logo of Datadog.


### LLMs using tools

There are also built-in tools that are part of the OpenAI platform. These tools enable the model to search the web, execute code, access the functionality of an MCP server, and more.

<p align="center">
    <img src="..\assets\openai-sdk\tools.png" width=400px>
</p>

For more info view [here](https://platform.openai.com/docs/guides/tools). 

Example below web search is one of the built in tools.

In [9]:
# Using web search tool
response = client.responses.create(
    model = "gpt-4o",
    tools = [{"type": "web_search_preview"}],
    input = "What is the latest information on the stock market today?"
)

print(response.output_text)

As of the latest trading session on Friday, September 19, 2025, U.S. stock markets continued their upward trajectory, achieving new record highs across major indices.

**Market Performance:**

- **S&P 500:** The index rose 0.5% to close at 6,631.96, marking its 26th new high of the year. ([abcnews.go.com](https://abcnews.go.com/Business/wireStory/major-us-stock-indexes-fared-thursday-9182025-125716282?utm_source=openai))

- **Dow Jones Industrial Average:** Gained 0.3%, ending at 46,142.42. ([abcnews.go.com](https://abcnews.go.com/Business/wireStory/major-us-stock-indexes-fared-thursday-9182025-125716282?utm_source=openai))

- **Nasdaq Composite:** Increased by 0.9%, closing at 22,470.73. ([abcnews.go.com](https://abcnews.go.com/Business/wireStory/major-us-stock-indexes-fared-thursday-9182025-125716282?utm_source=openai))

- **Russell 2000:** The small-cap index surged 2.5% to 2,467.70, reaching its first record high since November 2021. ([abcnews.go.com](https://abcnews.go.com/Busines

Going to now do file search using OpenAI platform storage

Enable the model to access information from a document repository. To enable vector search: 

1. Go to storage under your OpenAI dashboard [here](https://platform.openai.com/storage/)
2. Press on Vector Stores, create one, and add files.
3. Copy the Vector Store ID

In [12]:
response = client.responses.create(
    model = "gpt-4o-mini",
    input = "Compare the income from cba and macquarie for the year 2025",
    tools = [{
        "type": "file_search",
        "vector_store_ids": [os.getenv("bank_vector_id")]
    }]
)

print(response, "\n \n")
print(response.output_text, "\n \n")

Response(id='resp_68cf9113d9b88196a3ecb36e24ab1a600aba0124dc493e2c', created_at=1758433555.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseFileSearchToolCall(id='fs_68cf9114f9488196bfb24eac00c8d6090aba0124dc493e2c', queries=['income from CBA in 2025', 'income from Macquarie in 2025', 'CBA Macquarie income comparison 2025'], status='completed', type='file_search_call', results=None), ResponseOutputMessage(id='msg_68cf9117699081969ecc0871500307a10aba0124dc493e2c', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-W2dtuYpamg3WRW3VnCmxu8', filename='2025-cba-annual-report.pdf', index=160, type='file_citation'), AnnotationFileCitation(file_id='file-W2dtuYpamg3WRW3VnCmxu8', filename='2025-cba-annual-report.pdf', index=297, type='file_citation'), AnnotationFileCitation(file_id='file-HqeQ4iX4DF8NxVVravFGqj', filename='2025-macquarie-annual-report.pdf', index=423, type='file_ci

### Combining the tools + multimodal to test

In [20]:
prompt ="Tell me the latest news about the company depicted in the logo"
image_url =r"https://www.datocms-assets.com/104397/1714381237-datadog-logo-with-white-background.png"
# This photo is the datadog logo

response = client.responses.create(
    model = "gpt-4o", #only certain models do multimodal inputs
    input = [
        {"role": "user",
         "content": [
             {"type": "input_text", "text": prompt},
             {"type": "input_image", "image_url": image_url}
            ]
        }
    ],
    tools=[
        {"type":"web_search"}
    ]
)

print(response.output_text)

The logo you uploaded is associated with Datadog. Let me find the latest news about the company for you. Please hold on.Here’s a thorough and up-to-date overview of the **most recent news** about Datadog (the company represented by the logo you shared), current as of **September 21, 2025**, with precise dates and well-cited information:

---

##  Financial Performance & Strategic Highlights

### Second Quarter 2025 (Ended June 30, 2025)
- **Revenue jumped 28% year-over-year to $827 million**, showcasing robust growth ([investors.datadoghq.com](https://investors.datadoghq.com/news-releases/news-release-details/datadog-announces-second-quarter-2025-financial-results?utm_source=openai)).
- **Operating cash flow** reached **$200 million**, and **free cash flow** was **$165 million** ([investors.datadoghq.com](https://investors.datadoghq.com/news-releases/news-release-details/datadog-announces-second-quarter-2025-financial-results?utm_source=openai)).
- **Non-GAAP operating income** came in

In [22]:
import json
print(json.dumps(response.__dict__, default=lambda o: o.__dict__, indent=4))

{
    "id": "resp_68cf95f38c08819db207bb1e5816021608fee1786d5aa355",
    "created_at": 1758434803.0,
    "error": null,
    "incomplete_details": null,
    "instructions": null,
    "metadata": {},
    "model": "gpt-4o-2024-08-06",
    "object": "response",
    "output": [
        {
            "id": "msg_68cf95f508a0819da8f04da647749bdd08fee1786d5aa355",
            "content": [
                {
                    "annotations": [],
                    "text": "The logo you uploaded is associated with Datadog. Let me find the latest news about the company for you. Please hold on.",
                    "type": "output_text",
                    "logprobs": []
                }
            ],
            "role": "assistant",
            "status": "completed",
            "type": "message"
        },
        {
            "id": "ws_68cf95f65814819d95fddcaa65392c4308fee1786d5aa355",
            "action": {
                "query": "latest news Datadog 2023",
                "type": "sea